### IBM Data Science Capstone Project
#### The Battle of Neighborhoods

Import necessary packages

In [5]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import numpy as np
import json 
from geopy.geocoders import Nominatim 
from pandas.io.json import json_normalize
import matplotlib.cm as cm
import matplotlib.colors as colors
from sklearn.cluster import KMeans
import folium

print('Libraries imported.')

Libraries imported.


Define function to turn BeautifulSoup table into a python list

In [6]:
def tableDataText(table):       
    rows = []
    trs = table.find_all('tr')
    headerow = [td.get_text(strip=True) for td in trs[0].find_all('th')] # header row
    if headerow: # if there is a header row include first
        rows.append(headerow)
        trs = trs[1:]
    for tr in trs: # for every table row
        rows.append([td.get_text(strip=True) for td in tr.find_all('td')]) # data row
    return rows

Get data for towns within 30 miles of Rollinsford, NH and clean data

In [7]:
URL = "https://www.zip-codes.com/zip-code-radius-finder.asp?zipmileslow=0&zipmileshigh=30&zip1=03869&submit=Search"
page = requests.get(URL)
soup = BeautifulSoup(page.content, 'html.parser')
table = soup.find("div", {"id" : "tableview"})
list_table = tableDataText(table)
df = pd.DataFrame(list_table[1:], columns=list_table[0])
df = df.drop('#', axis = 1)
df = df[:-1]
df.tail()

,Zip,City,County,St,Country,Distance,
98,04061,NORTH WATERBORO,YORK,ME,US,29.18 miles,
99,03810,ALTON BAY,BELKNAP,NH,US,29.38 miles,
100,04006,BIDDEFORD POOL,YORK,ME,US,29.53 miles,
101,04056,NEWFIELD,YORK,ME,US,29.56 miles,
102,04063,OCEAN PARK,YORK,ME,US,29.60 miles,


Define function to get income data from zipatlas webpage across all pages

In [20]:
def getIncomeData(base_URL, num_pages):
    df_income = pd.DataFrame()
    i = 0
    while i<num_pages:
        if i == 0:
            URL = base_URL.format('')
        else:
            URL = base_URL.format(str(i+1) + ".")
        page = requests.get(URL)
        soup = BeautifulSoup(page.content, 'html.parser')
        table = soup.find('div', {'id' : 'ctl00_ContentPlaceHolder1_final_content'})
        income_table = tableDataText(table)
        df_income = df_income.append(pd.DataFrame(income_table[4:], columns=income_table[3]))
        i += 1
    return df_income[:-3]

Get income data for all towns in NH and ME, combine, and clean data

In [9]:
me_income = getIncomeData("http://zipatlas.com/us/me/zip-code-comparison/median-household-income.{}htm", 4)
nh_income = getIncomeData("http://zipatlas.com/us/nh/zip-code-comparison/median-household-income.{}htm", 3)
income_data = nh_income.append(me_income)
income_data[['Latitude', 'Longitude']] = income_data['Location'].str.split(',', expand = True)
income_data = income_data.drop(['#', 'Location', 'National Rank'], axis = 1)
income_data = income_data.rename(columns = {'Avg. Income/H/hold': 'Household Income'})
income_data.head()

,Zip Code,City,Population,Household Income,Latitude,Longitude
0,03087,"Windham,New Hampshire","10,709","$94,794.00",42.811092,-71.302688
1,03049,"Hollis,New Hampshire","7,017","$92,847.00",42.749626,-71.585401
2,03031,"Amherst,New Hampshire","10,812","$88,671.00",42.874854,-71.600512
3,03750,"Etna,New Hampshire",962,"$86,421.00",43.708188,-72.195047
4,03233,"Elkins,New Hampshire",234,"$85,247.00",43.438900,-71.952889


Join original dataframe with income data dataframe, split location col to lat & lon, and clean

In [10]:
df = pd.merge(df, income_data,how="inner", left_on="Zip", right_on = "Zip Code")
df = df.drop(['City_x', 'County', 'St', 'Country', 'Zip'], axis = 1)
df = df.rename(columns = {'City_y': 'City'})
df['Latitude'] = pd.to_numeric(df['Latitude'])
df['Longitude'] = pd.to_numeric(df['Longitude'])
df.head()

,Distance,,Zip Code,City,Population,Household Income,Latitude,Longitude
0,0.00 miles,,03869,"Rollinsford,New Hampshire","2,407","$48,380.00",43.219845,-70.843112
1,3.20 miles,,03878,"Somersworth,New Hampshire","11,483","$43,177.00",43.253476,-70.887944
2,3.52 miles,,03820,"Dover,New Hampshire","28,537","$44,493.00",43.187186,-70.902839
3,5.29 miles,,03908,"South Berwick,Maine","6,682","$53,294.00",43.230492,-70.752426
4,5.83 miles,,03901,"Berwick,Maine","6,338","$44,519.00",43.293960,-70.836455


Get lat and lon of Rollinsford to center map on, and plot surrounding towns with popups for location details

In [11]:
rollinsford = df[df['City'] == 'Rollinsford,New Hampshire']
map_rollinsford = folium.Map(location = [rollinsford['Latitude'], rollinsford['Longitude']], zoom_start = 9)

for lat, lng, city in zip(df['Latitude'], df['Longitude'], df['City']):
    label = '{}'.format(city)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_rollinsford)  

map_rollinsford

FourSquare Developer API access information

In [12]:
CLIENT_ID = 'W02GLHQCAYNTW4FFRCROILCNH5DPQPBQOT2RFBQCX4GNLERQ' # your Foursquare ID
CLIENT_SECRET = 'T5NZATV1DSRDNOTCQPVJTMVN5SWZRZHB4KE4MFW0PZMFPJRZ' # your Foursquare Secret
ACCESS_TOKEN = 'TTU5B5RAIMXLGMKTTL0YW2K0QBJJLKYYP2MMP0JQ3QCCGV3Y' # your FourSquare Access Token
VERSION = '20180604'
print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: W02GLHQCAYNTW4FFRCROILCNH5DPQPBQOT2RFBQCX4GNLERQ
CLIENT_SECRET:T5NZATV1DSRDNOTCQPVJTMVN5SWZRZHB4KE4MFW0PZMFPJRZ


Setting default radius (in m from given lat/lon) and limit of venues to get data on from FourSquare

In [13]:
radius = 3000
LIMIT = 500

Define functions to retrieve category types from venue data and to get all venues from FourSquare given locations

In [14]:
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']
    
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['City', 
                             'City Latitude', 
                             'City Longitude', 
                             'Venue', 
                             'Venue Latitude', 
                             'Venue Longitude', 
                             'Venue Category']
    
    return(nearby_venues)

Get all nearby venues in each of the towns provided

In [15]:
nearby_venues = getNearbyVenues(names = df['City'],
                                latitudes = df['Latitude'],
                                longitudes = df['Longitude'],
                                radius = 3000)
nearby_venues.shape

(1472, 7)

Perform one-hot encoding on venue data for further data analysis

In [16]:
venue_onehot = pd.get_dummies(nearby_venues[['Venue Category']], prefix = "", prefix_sep="")
venue_onehot['City'] = nearby_venues['City']

tmp = venue_onehot.pop('City')
venue_onehot.insert(0, 'City', tmp)

city_venues = venue_onehot.groupby('City').mean().reset_index()
city_venues.head()

,City,Accessories Store,Airport,Airport Terminal,American Restaurant,Antique Shop,Arcade,Art Gallery,Arts & Crafts Store,Asian Restaurant,...,Video Game Store,Video Store,Vineyard,Warehouse Store,Weight Loss Center,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio
0,"Acton,Maine",0.0,0.0,0.2,0.0,0.0,0.0,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,"Alfred,Maine",0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,"Alton Bay,New Hampshire",0.0,0.0,0.0,0.0,0.0,0.0,0.083333,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,"Alton,New Hampshire",0.0,0.0,0.0,0.2,0.0,0.0,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,"Barnstead,New Hampshire",0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


Print list of all venues found to identify possible positive/negative influences for our new location

In [17]:
print(list(city_venues.columns))

['City', 'Accessories Store', 'Airport', 'Airport Terminal', 'American Restaurant', 'Antique Shop', 'Arcade', 'Art Gallery', 'Arts & Crafts Store', 'Asian Restaurant', 'Assisted Living', 'Athletics & Sports', 'Auto Workshop', 'Automotive Shop', 'BBQ Joint', 'Bagel Shop', 'Bakery', 'Bank', 'Bar', 'Baseball Field', 'Basketball Court', 'Basketball Stadium', 'Beach', 'Bed & Breakfast', 'Beer Garden', 'Big Box Store', 'Bistro', 'Board Shop', 'Boat Launch', 'Boat or Ferry', 'Bookstore', 'Border Crossing', 'Boutique', 'Bowling Alley', 'Breakfast Spot', 'Brewery', 'Bridge', 'Burger Joint', 'Bus Station', 'Business Service', 'Butcher', 'Café', 'Campground', 'Candy Store', 'Caribbean Restaurant', 'Chinese Restaurant', 'Chocolate Shop', 'Clothing Store', 'Cocktail Bar', 'Coffee Shop', 'College Cafeteria', 'College Library', 'Comic Shop', 'Construction & Landscaping', 'Convenience Store', 'Cosmetics Shop', 'Dance Studio', 'Deli / Bodega', 'Department Store', 'Diner', 'Disc Golf', 'Discount Store',

Set good (positive) and bad (negative) venues for our location

In [21]:
good_cols = ['Athletics & Sports', 'Dance Studio','Golf Course','Gymnastics Gym', 'Juice Bar','Sporting Goods Shop','Soccer Field','Trail','Weight Loss Center','Yoga Studio']
bad_cols = ['Gym','Gym / Fitness Center']

Sum all towns 'good' and 'bad' columns to get proportion of venues falling into each category, clean and display by 'good'

In [22]:
city_venues['Good columns'] = city_venues[good_cols].sum(axis = 1)
city_venues['Bad columns'] = city_venues[bad_cols].sum(axis = 1)
sorted_city_venues = city_venues.sort_values(by='Good columns', ascending = False)
sorted_city_venues.head()

,City,Accessories Store,Airport,Airport Terminal,American Restaurant,Antique Shop,Arcade,Art Gallery,Arts & Crafts Store,Asian Restaurant,...,Vineyard,Warehouse Store,Weight Loss Center,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio,Good columns,Bad columns
67,"Suncook,New Hampshire",0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.500000,0.0
37,"Milton Mills,New Hampshire",0.0,0.0,0.0,0.0,0.333333,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.333333,0.0
47,"Nottingham,New Hampshire",0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.333333,0.0
56,"Saco,Maine",0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.250000,0.0
65,"Strafford,New Hampshire",0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.250000,0.0


In [ ]:
df_good_bad_cols = sorted_city_venues[['City','Good columns', 'Bad columns']]
df_info = pd.merge(df, df_good_bad_cols,how="inner", on = 'City')
df_info.head()

In [ ]:
df_info.sort_values('Good columns', ascending = False)

In [66]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 76 entries, 0 to 75
Data columns (total 8 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Distance          76 non-null     object 
 1                     76 non-null     object 
 2   Zip Code          76 non-null     object 
 3   City              76 non-null     object 
 4   Population        76 non-null     object 
 5   Household Income  76 non-null     object 
 6   Latitude          76 non-null     float64
 7   Longitude         76 non-null     float64
dtypes: float64(2), object(6)
memory usage: 7.4+ KB


In [65]:
nh_town_geo = 'https://earthworks.stanford.edu/download/file/massgis-nhtowns-poly-geojson.json'
nh_map = folium.Map(location = [rollinsford['Latitude'], rollinsford['Longitude']], zoom_start = 9)

folium.Choropleth(
    geo_data = nh_town_geo,
    data = df,
    columns = ['City', 'Household Income'],
    name = 'choropleth',
    key_on= 'features.properties.NAME',
    fill_color="YlGn",
    fill_opacity=0.7,
    line_opacity=0.2
                    ).add_to(nh_map)

nh_map

TypeError: ufunc 'isnan' not supported for the input types, and the inputs could not be safely coerced to any supported types according to the casting rule ''safe''

In [60]:
df_choro = df.rename(columns = {'City': 'NAME'})
df_choro['NAME'] = df_choro['NAME'].str.split(',', expand = True)

In [59]:
df_choro['NAME'].str.split(',')[0][0]

'Rollinsford'

In [61]:
df_choro.head()

,Distance,,Zip Code,NAME,Population,Household Income,Latitude,Longitude
0,0.00 miles,,03869,Rollinsford,"2,407","$48,380.00",43.219845,-70.843112
1,3.20 miles,,03878,Somersworth,"11,483","$43,177.00",43.253476,-70.887944
2,3.52 miles,,03820,Dover,"28,537","$44,493.00",43.187186,-70.902839
3,5.29 miles,,03908,South Berwick,"6,682","$53,294.00",43.230492,-70.752426
4,5.83 miles,,03901,Berwick,"6,338","$44,519.00",43.293960,-70.836455


In [35]:
import urllib.request
with urllib.request.urlopen(nh_town_geo) as url:
    data = json.loads(url.read().decode())
    print(data)

{'type': 'FeatureCollection', 'totalFeatures': 259, 'features': [{'type': 'Feature', 'id': 'GISDATA.NHTOWNS_POLY.11', 'geometry': {'type': 'MultiPolygon', 'coordinates': [[[[-71.32948054859314, 44.81977110797835], [-71.32812206122132, 44.783801752050536], [-71.32639183577795, 44.735571450438854], [-71.32548398140506, 44.70678315681801], [-71.21097365824001, 44.70985401477179], [-71.21550426748898, 44.826410360759375], [-71.22386926400297, 44.82633328812387], [-71.28904728242377, 44.824795944617165], [-71.32952265219177, 44.82335345611146], [-71.32948054859314, 44.81977110797835]]]]}, 'geometry_name': 'SHAPE', 'properties': {'FIPS': 7140, 'NAME': 'Millsfield', 'bbox': [-71.32952265219177, 44.70678315681801, -71.21097365824001, 44.826410360759375]}}, {'type': 'Feature', 'id': 'GISDATA.NHTOWNS_POLY.12', 'geometry': {'type': 'MultiPolygon', 'coordinates': [[[[-71.35290636113052, 44.8234620150237], [-71.36011063217617, 44.79683125792468], [-71.36265371400582, 44.78648706821973], [-71.328122